# SPRM Demo
### v1.5.6 December 18, 2025

## Setup Environment


Use setup.py to install all dependencies. 

Python => 3.8.

In [ ]:
!python ../setup.py install

# Download Example Image and Mask

You can use your own image and mask file but we provide these in case you want a quick look at SPRM features.

Download the corresponding mask and image files and put them in their respective img/ and mask/ directories.

https://drive.google.com/drive/folders/1denyZ1SFoWpWrPO9UbSdcF2DvHEv6ovN?usp=sharing

## Run SPRM

You'll be running a sample image and mask 

Est. Runtime: < 5 mins.

In [ ]:
!sh run_sprm.sh

## Display SPRM Results

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import IFrame
import pandas as pd
#import matplotlib
#%matplotlib inline
def dispclustersandlegend(png,csv=None):
    image = Image.open(png+".png")
    plt.figure()
    plt.imshow(image)
    plt.title(png)
    plt.axis('off')
    plt.show()
    if csv is not None:
        legend = pd.read_csv(csv+".csv")
        print(legend)

In [ ]:
folder_path = "sprm_demo_outputs/"
input_file = folder_path+"image_demo.ome.tiff"

## Show image quality metrics

In [ ]:
import json
with open(input_file + "-SPRM_Image_Quality_Measures.json", "r") as f:
    jsoncontent = json.load(f)
print(json.dumps(jsoncontent,indent=2))

## Pixels colored by the first three principal components of the pixel intensities 
### Table below shows the channels most distinguishing the clusters and their weightings

In [ ]:
dispclustersandlegend(input_file+"-Top3ChannelPCA",input_file+"-channelPCA_summary")

## Cell Adjancy Graph shows lines between cells that are within shown distance (in pixels) of each other

In [ ]:
import base64
with open(input_file + "_AdjacencyGraph.pdf", "rb") as pdf:
    content = pdf.read()
# encode PDF
base64_pdf = base64.b64encode(content).decode("utf-8")
# display encoded PDF
IFrame(f"data:application/pdf;base64,{base64_pdf}", width=600, height=400)

## Cells colored by cell cluster after clustering using UMAP embedding of all features

In [ ]:
dispclustersandlegend(input_file+"-clusterbyUMAP")

## Cells colored by cell cluster using different feature sets of whole cells
### Tables below each image show the channels most distinguishing the clusters and their average feature value for each cluster (covariance features are correlations between two channels)

In [ ]:
dispclustersandlegend(input_file+"-clusterbyMeanspercell",input_file+"-clustercell_mean_legend")
dispclustersandlegend(input_file+"-clusterbyTotalpercell",input_file+"-clustercell_total_legend")
dispclustersandlegend(input_file+"-clusterbyCovarpercell",input_file+"-clustercell_covariance_legend")

## Cells colored by cell cluster using different feature sets of nuclei
### Tables below each image show the channels most distinguishing the clusters and their average feature value for each cluster (covariance features are correlations between two channels)

In [ ]:
dispclustersandlegend(input_file+"-clusterbyMeanspernuclei",input_file+"-clusternuclei_mean_legend")
dispclustersandlegend(input_file+"-clusterbyTotalpernuclei",input_file+"-clusternuclei_total_legend")
dispclustersandlegend(input_file+"-clusterbyCovarpernuclei",input_file+"-clusternuclei_covariance_legend")

## Cells colored by cell cluster using cell shape+size or normalized shape

In [ ]:
dispclustersandlegend(input_file+"-cluster_Shape")
dispclustersandlegend(input_file+"-cluster_ShapeNormalized")

## The cell centers, cell features (in separate files by feature type), and the cluster assignments for each cell (only for those not ignored because it was too small or touching the image edge) are contained in CSV files
### The first 5 rows of each CSV are shown

In [ ]:
import pandas as pd
cellcenters = pd.read_csv(input_file + "-cell_centers.csv")
cellcenters.head()

In [ ]:
cellchannelmean = pd.read_csv(input_file + "-cell_channel_mean.csv")
cellchannelmean.head()

In [ ]:
cellclusters = pd.read_csv(input_file + "-cell_cluster.csv")
cellclusters.head()